In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [3]:
#data_snp_phenotype = pd.read_csv('../data/data_snp_phenotype_012.csv', sep=',')
data_snp_phenotype = pd.read_pickle('../data/data.imputed_geno_groups_subgroups.pkl.gz')


In [4]:
data_snp_phenotype = data_snp_phenotype.drop(columns=['Subgroup'])

In [7]:
data_snp_phenotype = data_snp_phenotype.iloc[:, :5000]

In [8]:
data_snp_phenotype

,Heterotic group,NC_024459.2:53013,NC_024459.2:53064,NC_024459.2:53109,NC_024459.2:56073,NC_024459.2:262740,NC_024459.2:262761,NC_024459.2:262785,NC_024459.2:262790,NC_024459.2:262793,...,NC_024459.2:34509007,NC_024459.2:34509043,NC_024459.2:34509048,NC_024459.2:34540632,NC_024459.2:34540635,NC_024459.2:34540659,NC_024459.2:34540688,NC_024459.2:34540704,NC_024459.2:34540728,NC_024459.2:34541157
0,NSS,A|G,C|C,C|T,G|C,C|C,C|A,A|C,T|C,T|C,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
1,SPT,A|G,C|C,C|T,C|C,C|C,C|A,A|C,T|C,T|C,...,G|G,G|G,C|C,A|G,A|G,C|C,C|C,A|C,G|G,C|C
2,NSS,A|G,T|C,C|T,C|C,C|C,C|A,A|C,T|C,T|C,...,G|G,A|G,C|C,G|G,G|G,C|C,C|C,C|C,G|C,C|C
3,PB,A|G,T|C,C|T,G|G,T|T,C|C,A|A,T|T,T|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
4,PA,A|A,C|C,C|C,C|C,C|C,C|A,A|C,T|C,T|C,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,PA,A|G,C|C,C|T,C|C,C|C,C|A,A|C,T|C,T|C,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
601,PA,A|G,C|C,C|T,C|C,C|C,C|A,A|C,T|C,T|C,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
602,PA,A|G,C|C,C|T,C|C,C|C,C|A,A|C,T|C,T|C,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
603,PA,A|G,T|C,C|T,G|G,T|T,C|C,A|A,T|T,T|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C


In [9]:
def standardize_genotypes(data_snp_phenotype):
    for col in data_snp_phenotype.columns[1:]:
        data_snp_phenotype.loc[:, col] = data_snp_phenotype[col].apply(lambda x: '|'.join(sorted(x.split('|'))))

    return data_snp_phenotype

standardize_genotypes(data_snp_phenotype)

,Heterotic group,NC_024459.2:53013,NC_024459.2:53064,NC_024459.2:53109,NC_024459.2:56073,NC_024459.2:262740,NC_024459.2:262761,NC_024459.2:262785,NC_024459.2:262790,NC_024459.2:262793,...,NC_024459.2:34509007,NC_024459.2:34509043,NC_024459.2:34509048,NC_024459.2:34540632,NC_024459.2:34540635,NC_024459.2:34540659,NC_024459.2:34540688,NC_024459.2:34540704,NC_024459.2:34540728,NC_024459.2:34541157
0,NSS,A|G,C|C,C|T,C|G,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
1,SPT,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,C|C,A|G,A|G,C|C,C|C,A|C,G|G,C|C
2,NSS,A|G,C|T,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,A|G,C|C,G|G,G|G,C|C,C|C,C|C,C|G,C|C
3,PB,A|G,C|T,C|T,G|G,T|T,C|C,A|A,T|T,T|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
4,PA,A|A,C|C,C|C,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,PA,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
601,PA,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
602,PA,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
603,PA,A|G,C|T,C|T,G|G,T|T,C|C,A|A,T|T,T|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C


In [10]:
data_snp_phenotype

,Heterotic group,NC_024459.2:53013,NC_024459.2:53064,NC_024459.2:53109,NC_024459.2:56073,NC_024459.2:262740,NC_024459.2:262761,NC_024459.2:262785,NC_024459.2:262790,NC_024459.2:262793,...,NC_024459.2:34509007,NC_024459.2:34509043,NC_024459.2:34509048,NC_024459.2:34540632,NC_024459.2:34540635,NC_024459.2:34540659,NC_024459.2:34540688,NC_024459.2:34540704,NC_024459.2:34540728,NC_024459.2:34541157
0,NSS,A|G,C|C,C|T,C|G,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
1,SPT,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,C|C,A|G,A|G,C|C,C|C,A|C,G|G,C|C
2,NSS,A|G,C|T,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,A|G,C|C,G|G,G|G,C|C,C|C,C|C,C|G,C|C
3,PB,A|G,C|T,C|T,G|G,T|T,C|C,A|A,T|T,T|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
4,PA,A|A,C|C,C|C,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,PA,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
601,PA,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
602,PA,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
603,PA,A|G,C|T,C|T,G|G,T|T,C|C,A|A,T|T,T|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C


In [11]:
df_wo_hetero_group = data_snp_phenotype.drop(columns=['Heterotic group'])
all_values = df_wo_hetero_group.values.flatten()
unique_values = pd.unique(all_values)
unique_values

array(['A|G', 'C|C', 'C|T', 'C|G', 'A|C', 'T|T', 'G|G', 'A|T', 'A|A',
       'G|T'], dtype=object)

In [12]:
# Divide the data into X as a data for learning models aka features and y as a data the model should predict aka target
X = data_snp_phenotype.drop(columns=["Heterotic group"]) # Генотипы
y = data_snp_phenotype["Heterotic group"]  # гетерозисная группа

In [13]:
X

,NC_024459.2:53013,NC_024459.2:53064,NC_024459.2:53109,NC_024459.2:56073,NC_024459.2:262740,NC_024459.2:262761,NC_024459.2:262785,NC_024459.2:262790,NC_024459.2:262793,NC_024459.2:262918,...,NC_024459.2:34509007,NC_024459.2:34509043,NC_024459.2:34509048,NC_024459.2:34540632,NC_024459.2:34540635,NC_024459.2:34540659,NC_024459.2:34540688,NC_024459.2:34540704,NC_024459.2:34540728,NC_024459.2:34541157
0,A|G,C|C,C|T,C|G,C|C,A|C,A|C,C|T,C|T,C|T,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
1,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,C|T,...,G|G,G|G,C|C,A|G,A|G,C|C,C|C,A|C,G|G,C|C
2,A|G,C|T,C|T,C|C,C|C,A|C,A|C,C|T,C|T,C|T,...,G|G,A|G,C|C,G|G,G|G,C|C,C|C,C|C,C|G,C|C
3,A|G,C|T,C|T,G|G,T|T,C|C,A|A,T|T,T|T,C|C,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
4,A|A,C|C,C|C,C|C,C|C,A|C,A|C,C|T,C|T,C|T,...,G|G,G|G,C|C,G|G,G|G,T|T,C|C,C|C,G|G,C|C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
601,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
602,A|G,C|C,C|T,C|C,C|C,A|C,A|C,C|T,C|T,C|T,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C
603,A|G,C|T,C|T,G|G,T|T,C|C,A|A,T|T,T|T,C|C,...,G|G,G|G,T|T,G|G,G|G,T|T,C|C,C|C,G|G,C|C


In [14]:
y

0      NSS
1      SPT
2      NSS
3       PB
4       PA
      ... 
600     PA
601     PA
602     PA
603     PA
604    NSS
Name: Heterotic group, Length: 604, dtype: object

In [15]:
def genotype_to_numeric(genotype, heterozygote):
    alleles = genotype.split('|')
    
    if alleles[0] != alleles[1]:
        return 0
    if alleles[0] == heterozygote[0]:
        return 1
    elif alleles[0] == heterozygote[1]:
        return 2

    return -1

for column in X.columns:
    heterozygote = None
    first_genotype = None  
    for idx, genotype in enumerate(X[column]):
        alleles = genotype.split('|')
        
        if alleles[0] == alleles[1]:
            if first_genotype is None:  
                first_genotype = genotype
        
        if alleles[0] != alleles[1]:
            heterozygote = alleles
            break

    if heterozygote is None:

        X[column] = X[column].apply(lambda x: 1 if x == first_genotype else 2)
    else:
        X[column] = X[column].apply(lambda x: genotype_to_numeric(x, heterozygote))

X


,NC_024459.2:53013,NC_024459.2:53064,NC_024459.2:53109,NC_024459.2:56073,NC_024459.2:262740,NC_024459.2:262761,NC_024459.2:262785,NC_024459.2:262790,NC_024459.2:262793,NC_024459.2:262918,...,NC_024459.2:34509007,NC_024459.2:34509043,NC_024459.2:34509048,NC_024459.2:34540632,NC_024459.2:34540635,NC_024459.2:34540659,NC_024459.2:34540688,NC_024459.2:34540704,NC_024459.2:34540728,NC_024459.2:34541157
0,0,1,0,0,1,0,0,0,0,0,...,2,2,1,2,2,2,1,2,2,1
1,0,1,0,1,1,0,0,0,0,0,...,2,2,1,0,0,1,1,0,2,1
2,0,0,0,1,1,0,0,0,0,0,...,2,0,1,2,2,1,1,2,0,1
3,0,0,0,2,2,2,1,2,2,1,...,2,2,2,2,2,2,1,2,2,1
4,1,1,1,1,1,0,0,0,0,0,...,2,2,1,2,2,2,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,0,1,0,1,1,0,0,0,0,0,...,2,2,2,2,2,2,1,2,2,1
601,0,1,0,1,1,0,0,0,0,0,...,2,2,2,2,2,2,1,2,2,1
602,0,1,0,1,1,0,0,0,0,0,...,2,2,2,2,2,2,1,2,2,1
603,0,0,0,2,2,2,1,2,2,1,...,2,2,2,2,2,2,1,2,2,1


In [16]:
X.dtypes

NC_024459.2:53013       int64
NC_024459.2:53064       int64
NC_024459.2:53109       int64
NC_024459.2:56073       int64
NC_024459.2:262740      int64
                        ...  
NC_024459.2:34540659    int64
NC_024459.2:34540688    int64
NC_024459.2:34540704    int64
NC_024459.2:34540728    int64
NC_024459.2:34541157    int64
Length: 4999, dtype: object

In [17]:
X = X.astype('int8')


In [18]:
X.dtypes

NC_024459.2:53013       int8
NC_024459.2:53064       int8
NC_024459.2:53109       int8
NC_024459.2:56073       int8
NC_024459.2:262740      int8
                        ... 
NC_024459.2:34540659    int8
NC_024459.2:34540688    int8
NC_024459.2:34540704    int8
NC_024459.2:34540728    int8
NC_024459.2:34541157    int8
Length: 4999, dtype: object

In [19]:
X.shape

(604, 4999)

In [20]:
y.shape

(604,)

In [21]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [22]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [23]:
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    cat_features=X.columns.tolist(),  
    verbose=100,
    random_seed=42,
    early_stopping_rounds=50,
    used_ram_limit='32gb',
    thread_count=1
)

In [24]:
model.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),
    use_best_model=True,
    verbose=10
)

0:	learn: 0.4202899	test: 0.4166667	best: 0.4166667 (0)	total: 24s	remaining: 39m 31s
10:	learn: 0.6024845	test: 0.5500000	best: 0.5666667 (9)	total: 4m 28s	remaining: 36m 12s
20:	learn: 0.6811594	test: 0.6500000	best: 0.6500000 (20)	total: 8m 42s	remaining: 32m 44s
30:	learn: 0.7267081	test: 0.6666667	best: 0.6666667 (29)	total: 12m 55s	remaining: 28m 45s
40:	learn: 0.7536232	test: 0.6833333	best: 0.6833333 (33)	total: 17m 8s	remaining: 24m 39s
50:	learn: 0.7701863	test: 0.6833333	best: 0.6833333 (33)	total: 21m 12s	remaining: 20m 22s
60:	learn: 0.7867495	test: 0.6833333	best: 0.6833333 (33)	total: 25m 6s	remaining: 16m 2s
70:	learn: 0.7971014	test: 0.6833333	best: 0.6833333 (33)	total: 29m 5s	remaining: 11m 53s
80:	learn: 0.7971014	test: 0.7000000	best: 0.7000000 (75)	total: 33m	remaining: 7m 44s
90:	learn: 0.8074534	test: 0.7166667	best: 0.7333333 (86)	total: 37m 6s	remaining: 3m 40s
99:	learn: 0.8115942	test: 0.7333333	best: 0.7333333 (86)	total: 40m 55s	remaining: 0us

bestTest = 

CatBoost is using more CPU RAM (38.6GiB) than the limit (32GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(32536) > ResourceQuota(0)
Resource CPU

In [25]:
train_accuracy_filtered_data = accuracy_score(y_train, model.predict(X_train))
test_accuracy_filtered_data = accuracy_score(y_test, model.predict(X_test))
print(f"Train accuracy: {train_accuracy_filtered_data}")
print(f"Test accuracy: {test_accuracy_filtered_data}")

train_precision = precision_score(y_train, model.predict(X_train), average='weighted')
test_precision = precision_score(y_test, model.predict(X_test), average='weighted')
print(f"Train precision: {train_precision}")
print(f"Test precision: {test_precision}")

train_recall = recall_score(y_train, model.predict(X_train), average='weighted')
test_recall = recall_score(y_test, model.predict(X_test), average='weighted')
print(f"Train recall: {train_recall}")
print(f"Test recall: {test_recall}")

train_f1 = f1_score(y_train, model.predict(X_train), average='weighted')
test_f1 = f1_score(y_test, model.predict(X_test), average='weighted')
print(f"Train F1-score: {train_f1}")
print(f"Test F1-score: {test_f1}")

train_roc_auc = roc_auc_score(y_train, model.predict_proba(X_train), multi_class='ovr')
test_roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Test ROC AUC: {test_roc_auc}")

Train accuracy: 0.782608695652174
Test accuracy: 0.7377049180327869


/mnt/users/ib_2025/miniforge3/envs/corn_catboost/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train precision: 0.8125741721217417
Test precision: 0.7088778015363902
Train recall: 0.782608695652174
Test recall: 0.7377049180327869
Train F1-score: 0.7561777553187119
Test F1-score: 0.7072686537945636
Train ROC AUC: 0.9760607686199088
Test ROC AUC: 0.9335835362730945
